tb - 6/25/2022 - Testing the hypothesis that low-complexity models are more descriptive when phrased using climate-invariant inputs

# Imports and initialization

## Imports

In [1]:
from cbrain.climate_invariant import *
from cbrain.climate_invariant_utils import *
import pickle
#import h5netcdf
import time
import matplotlib as mpl
import matplotlib.pyplot as plt

from cbrain.imports import *
from cbrain.utils import *
from cbrain.normalization import *

import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_memory_growth(physical_devices[1], True)
tf.config.experimental.set_memory_growth(physical_devices[2], True)

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


## Initialization

In [2]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'
path_array = {}
climate_str = ['cold','hot','both']
set_str = ['train','valid','test']
test_clim_str = ['cold','hot','both','medium']

path_array['cold'] = [path_data+'2021_03_18_O3_TRAIN_M4K_shuffle.nc',
                      path_data+'2021_03_18_O3_VALID_M4K.nc',
                      path_data+'2021_03_18_O3_TEST_M4K.nc']
path_array['hot'] = [path_data+'2021_03_18_O3_TRAIN_P4K_shuffle.nc',
                     path_data+'2021_03_18_O3_VALID_P4K.nc',
                     path_data+'2021_03_18_O3_TEST_P4K.nc']
path_array['both'] = [path_data+'2022_04_18_TRAIN_M4K_P4K_shuffle.nc',
                      path_data+'2022_04_18_VALID_M4K_P4K.nc',
                      path_data+'2022_04_18_TEST_M4K_P4K.nc']
path_array['medium'] = [path_data+'2021_01_24_O3_TRAIN_shuffle.nc',
                        path_data+'2021_01_24_O3_VALID.nc',
                        path_data+'2021_01_24_O3_TEST.nc']

path_input_norm = path_data + '2021_01_24_NORM_O3_small.nc'
scale_dict = pickle.load(open(path_data+'009_Wm2_scaling.pkl','rb'))
path_norm_RH = path_data + '2021_02_01_NORM_O3_RH_small.nc'
scale_dict_RH = scale_dict.copy()
scale_dict_RH['RH'] = 0.01*L_S/G, # Arbitrary 0.1 factor as specific humidity is generally below 2%
path_train_RH = path_data + '2021_01_24_O3_small_shuffle.nc'
path_norm_BMSE = path_data + '2021_06_16_NORM_BMSE_small.nc'
path_train_BMSE = path_data + '2021_06_16_BMSE_small_shuffle.nc'
path_norm_LHF_nsDELQ = path_data + '2021_02_01_NORM_O3_LHF_nsDELQ_small.nc'
path_train_LHF_nsDELQ = path_data + '2021_02_01_O3_LHF_nsQ_small_shuffle.nc'

in_vars = ['QBP','TBP','PS','SOLIN','SHFLX','LHFLX'] # We take the large-scale climate state as inputs
out_vars = ['PHQ','TPHYSTND','QRL','QRS'] # and we output the response of clouds/storms to these climate conditions

In [3]:
fz = 12
lw = 2
siz = 100

plt.rc('text', usetex=False)
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
plt.rc('font', family='serif', size=fz)
mpl.rcParams['lines.linewidth'] = lw

# Data generators

In [4]:
def train_gen_rescaling(input_rescaling,path_norm,path_train,scale_dict):
    return DataGeneratorCI(
        data_fn = path_train,
        input_vars = input_rescaling,
        output_vars = out_vars,
        norm_fn = path_norm,
        input_transform = ('mean', 'maxrs'),
        output_transform = scale_dict)

In [5]:
train_gen_RH = train_gen_rescaling(['RH','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX'],
                                   path_norm_RH,path_train_RH,scale_dict_RH)
train_gen_BMSE = train_gen_rescaling(['QBP','BMSE','PS', 'SOLIN', 'SHFLX', 'LHFLX'],
                                     path_norm_BMSE,path_train_BMSE,scale_dict)
train_gen_LHF_nsDELQ = train_gen_rescaling(['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHF_nsDELQ'],
                                           path_norm_LHF_nsDELQ,path_train_LHF_nsDELQ,scale_dict)

In [6]:
def Generator_singleDS(path,rescaling=None):
    
    in_vars = ['QBP','TBP','PS','SOLIN','SHFLX','LHFLX'] # We take the large-scale climate state as inputs
    out_vars = ['PHQ','TPHYSTND','QRL','QRS'] # and we output the response of clouds/storms to these climate conditions
    path_input_norm = path_data + '2021_01_24_NORM_O3_small.nc'
    scale_dict = pickle.load(open(path_data+'009_Wm2_scaling.pkl','rb'))
    
    if rescaling=='CI':
        gen = DataGeneratorCI(
        data_fn = path,
        input_vars = in_vars,
        output_vars = out_vars,
        norm_fn = path_input_norm,
        batch_size=8192,
        input_transform = ('mean', 'maxrs'),
        output_transform = scale_dict,
        Qscaling = 'RH',
        Tscaling = 'BMSE',
        LHFscaling = 'LHF_nsDELQ',
        hyam=hyam, hybm=hybm, # Arrays to define mid-levels of hybrid vertical coordinate
        inp_sub_Qscaling=train_gen_RH.input_transform.sub, # What to subtract from RH inputs
        inp_div_Qscaling=train_gen_RH.input_transform.div, # What to divide RH inputs by
        inp_sub_Tscaling=train_gen_BMSE.input_transform.sub,
        inp_div_Tscaling=train_gen_BMSE.input_transform.div,
        inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
        inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div
        ) 
    else:
        gen = DataGeneratorCI(
        data_fn = path,
        input_vars = in_vars,
        output_vars = out_vars,
        norm_fn = path_input_norm,
        batch_size=8192,
        input_transform = ('mean', 'maxrs'),
        output_transform = scale_dict
        )

    return gen 

In [7]:
BFgen = {}
CIgen = {}

In [8]:
for iclimate,clim in enumerate(test_clim_str):
    print('Climate = ',clim)
    BFgen[clim] = {}
    CIgen[clim] = {}
    
    for iset,st in enumerate(set_str):
        print('Set = ',st)
        
        BFgen[clim][st] = Generator_singleDS(path_array[clim][iset])
        CIgen[clim][st] = Generator_singleDS(path_array[clim][iset],rescaling='CI')

Climate =  cold
Set =  train
Set =  valid
Set =  test
Climate =  hot
Set =  train
Set =  valid
Set =  test
Climate =  both
Set =  train
Set =  valid
Set =  test
Climate =  medium
Set =  train
Set =  valid
Set =  test


# Model 1: Multiple Linear Regression

Steps:    
- Train on training set containing both climates    
- Evaluate on test set of each climate separately   

## Model definition

In [9]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
MLR_BF = tf.keras.models.Model(inp, dense_out)

In [10]:
inp2 = Input(shape=(64,)) ## input after rh and tns transformation
dense2 = Dense(120, activation='linear')(inp2)
MLR_CI = tf.keras.models.Model(inp2, dense2)

In [11]:
MLR_BF.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [12]:
MLR_BF.compile(tf.keras.optimizers.Adam(), loss=mse)
MLR_CI.compile(tf.keras.optimizers.Adam(), loss=mse)

In [13]:
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2022_07_12_MLR_both'

In [14]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_BF = ModelCheckpoint(path_HDF5+save_name+'_BF.hdf5',save_best_only=True, monitor='val_loss', mode='min')
mcp_save_CI = ModelCheckpoint(path_HDF5+save_name+'_CI.hdf5',save_best_only=True, monitor='val_loss', mode='min')

## Model training

In [15]:
Nep = 20

In [16]:
# MLR_BF.fit_generator(BFgen['both']['train'], epochs=Nep, validation_data=BFgen['both']['valid'],
#                      callbacks=[earlyStopping, mcp_save_BF])

In [17]:
MLR_BF.load_weights(path_HDF5+'2022_06_25_MLR_both'+'_BF.hdf5')
#MLR_CI.load_weights(path_HDF5+save_name+'_CI.hdf5')

In [ ]:
MLR_CI.fit_generator(CIgen['both']['train'], epochs=Nep, validation_data=CIgen['both']['valid'],
                     callbacks=[earlyStopping, mcp_save_CI])

Epoch 1/20
11518/11518 [==============================] - 4028s 350ms/step - loss: 522.6601 - val_loss: 500.0511
Epoch 2/20
11518/11518 [==============================] - 3337s 290ms/step - loss: 493.4946 - val_loss: 490.3513
Epoch 3/20
11518/11518 [==============================] - 3461s 301ms/step - loss: 487.5516 - val_loss: 486.5635
Epoch 4/20
11518/11518 [==============================] - 3482s 302ms/step - loss: 484.5118 - val_loss: 484.2162
Epoch 5/20
11517/11518 [============================>.] - ETA: 0s - loss: 482.5082

## Model evaluation

In [ ]:
st = 'test'
BFtest = {}; CItest = {}

In [ ]:
for iclimate,clim in enumerate(test_clim_str):
    print('Climate = ',clim)
    BFg = BFgen[clim][st]; CIg = CIgen[clim][st]
    BFtest[clim] = MLR_BF.evaluate_generator(BFg)
    print('BF ',print(BFtest[clim]))
    CItest[clim] = MLR_CI.evaluate_generator(CIg)
    print('CI ',print(CItest[clim]))

In [ ]:
path_test = path_data + 'PKL_DATA/2022_07_12_Performance_Four_Climates_MLR_Both'

pickle.dump({'BFtest':BFtest,'CItest':CItest},open(path_test,'wb'))

In [ ]:
print(BFtest); print(CItest);

tb - 06/26/2022 - The results are negative once again. If allowed to be fully non-local and use data from both climates, climate-invariant mappings are no better than brute-force ones.

tb - 07/12/2022 - Trying again with improved CI data generator